In [ ]:
#!pip install python-barcode pillow

In [ ]:
import os
import sqlite3
from PIL import Image, ImageDraw, ImageFont
from barcode import Code128
from barcode.writer import ImageWriter

In [ ]:
#Conecta ao banco de dados SQLite
conn = sqlite3.connect('alunos.db')
cursor = conn.cursor()

#obtem os dados dos alunos
cursor.execute("SELECT Matricula, Nome, Serie, Turma FROM alunos")
students = cursor.fetchall()

#configura a fonte
font_path = "C:\\Windows\\Fonts\\arialbd.ttf"  # Caminho para a fonte Arial Bold (ajuste se necessário)
font_size = 20
font = ImageFont.truetype(font_path, font_size)

#adiciona os acentos nos nomes das colunas
column_names = ['Nome', 'Serie', 'Turma', 'Matricula']
display_names = {
    'Nome': 'Nome',
    'Serie': 'Série',
    'Turma': 'Turma',
    'Matricula': 'Matrícula'
}

#função pra redimensionar a foto
def resize_image(image, max_width, max_height):
    width_ratio = max_width / image.width
    height_ratio = max_height / image.height
    new_size = min(width_ratio, height_ratio)
    return image.resize((int(image.width * new_size), int(image.height * new_size)))

#criador de carteirinha
def create_student_card(student):
    card_width, card_height = 440, 250  #tamanho da carteirinha
    card = Image.new('RGB', (card_width, card_height), 'white')
    draw = ImageDraw.Draw(card)

    #Adiciona a foto
    try:
        photo_path = os.path.join('images', f"{student[0]}.jpg")
        photo = Image.open(photo_path)
        photo = resize_image(photo, 150, 200)  # tamanho da foto
        card.paste(photo, (10, 25))
    except Exception as e:
        print(f"Erro ao carregar foto para {student[0]}: {e}")

    # Adicionar informações do aluno a carteirinha
    text_color = 'black'
    line_height = 30
    y_position = 10
    x_text_position = 190  # posição do texto, deslocado pra direita
    for i, key in enumerate(column_names):
        draw.text((x_text_position, y_position), f"{display_names[key]}: {student[i]}", fill=text_color, font=font)
        y_position += line_height

    # gerador de código de barras
    try:
        writer_options = {
            'write_text': False
        }
        barcode_fileNome = f"barcode_{student[0]}"
        barcode = Code128(student[0], writer=ImageWriter())
        barcode.save(barcode_fileNome, options=writer_options)

        # verifica se deu certo
        barcode_fileNome = f"{barcode_fileNome}.png"
        if not os.path.exists(barcode_fileNome):
            raise FileNotFoundError(f"Arquivo {barcode_fileNome} não foi criado")

        # abre e corrige o tamanho
        barcode_img = Image.open(barcode_fileNome)
        barcode_width = 300 #tamanho do código de barra
        barcode_height = int(barcode_img.height * barcode_width / barcode_img.width)
        barcode_img = barcode_img.resize((barcode_width, barcode_height-140))
        card.paste(barcode_img, (160, y_position+10)) #posição do código de barra

        # deleta o arquivo temporário do codigo de barra
        os.remove(barcode_fileNome)
    except Exception as e:
        print(f"Erro ao gerar código de barras para {student[0]}: {e}")

    # Salvar a carteirinha como imagem JPG
    card_fileNome = os.path.join('cards', f"carteirinha_{student[0]}.jpg")
    card.save(card_fileNome)
    print(f"Cartão salvo como {card_fileNome}")

# itera sobre a lista e cria a carteirinha de todos os alunos
for student in students:
    create_student_card(student)

conn.close()
